# Tier Wrapper generator for clesperantoj / CLIJ3

The following notebook generates the wrapping code to make C++ available to the clesperantoj and CLIJ3 Java code.

__*WARNING*__: this can break the code if the java package is not update consequently. Please check the code and run the tests before pushing.

In [1]:
CLIC_DIR = 'C:/structure/code/CLIc_prototype/clic'

These files will be overwritten:

In [2]:
KERNELJ_HEADER_FILE = 'C:/structure/code/clesperantoj_prototype/native/clesperantoj/include/kernelj.hpp'

In [3]:
KERNELJ_SOURCE_FILE = 'C:/structure/code/clesperantoj_prototype/native/clesperantoj/src/kernelj.cpp'

In [4]:
CLIJ3_GATEWAY_SOURCE_FILE = 'C:/structure/code/clij3/src/main/java/net/clesperanto/CLIJ3.java'

These kernels will be excluded from the API:

In [5]:
GENERATOR_BLACKLIST =  ["Custom", "ExecuteSeparable", "Separable"]

These functions help deciding what output-type functions have.

In [6]:
def produces_label_image(function_name):
    return function_name in [
        "DilateLabels",
        "ErodeLabels",
        "ConnectedComponentLabelingBox",
        "MaskedVoronoiLabeling",
        "VoronoiOtsuLabeling"
    ]

def produces_binary_image(function_name):
    return (function_name.startswith("Binary") or 
           function_name.startswith("Threshold"))

def produces_x_projection(function_name):
    return function_name.endswith("XProjection")

def produces_y_projection(function_name):
    return function_name.endswith("YProjection")

def produces_z_projection(function_name):
    return function_name.endswith("ZProjection")

produces_binary_image("threshold_otsu")

False

## Parsing code and generating code-snippets
The following cells demonstrate how C++ code is parsed and how code is generated from that.

In [7]:
import glob
import os
import sys
import numpy as np

# define __file__ if it is not defined
if '__file__' not in globals():
    __file__ = os.path.abspath('generate-java.ipynb')

# get platform linux, windows, darwin
platform = sys.platform
# get platform architecture x86_64, x86, armv7l
try:
    architecture = os.uname().machine
except:
    architecture = ""
# get the python version
python_version = sys.version_info

Get the list of hpp files in the tiers directory and sort them by name.

In [8]:
# get first argument of the script
tier = 1 #sys.argv[1]

# get current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the relative path to directory and add 'tier' plus the number of the tier
tier_dir = os.path.join('C:/structure/code/CLIc_prototype/clic', 'include', 'tier' + str(tier))
# list all files in the tier_dir
files = glob.glob(tier_dir + '/*.hpp')
files.sort()
print(files)

['C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleAbsoluteKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleAddImageAndScalarKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleAddImagesWeightedKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinaryAndKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinaryEdgeDetectionKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinaryNotKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinaryOrKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinarySubtractKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBinaryXorKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleBlockEnumerateKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleConvolveKernel.hpp', 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleCopyKernel.hpp',

Define a set of function to parse the hpp files and extract the information we need.

In [9]:
file = 'C:/structure/code/CLIc_prototype/clic\\include\\tier1\\cleDivideImageAndScalarKernel.hpp'

In [10]:
def read_function_signature(file, function_name):
    with open(file, 'r') as f:
        # read the file content
        content = f.read()
        # find the position of the function of interest
        start = content.find(function_name + 'Kernel_Call')
        # find the position of the function signature
        start = content.find('(', start)
        # find the position of the end of the function signature
        end = content.find(')', start)
        # extract the function signature
        function_signature = content[start:end+1]
        # remove the prefix 'cle' from the function signature
        function_signature = function_signature.replace('cle', '')
        # remove the suffix 'Kernel_Call' from the function signature
        function_signature = function_signature.replace('Kernel_Call', '')
    return function_signature

function_name = os.path.basename(file)[3:-10]
function_signature = read_function_signature(file, function_name)
function_signature

'(const std::shared_ptr<::Processor> & device,\n                                const Image &                           src,\n                                const Image &                           dst,\n                                const float &                           scalar)'

In [11]:
def get_parameter_names(function_signature):
    # split the function signature into a list of parameters
    parameters = function_signature[1:-1].split(',')
    # get the parameter names
    parameter_names = [p.split(' ')[-1] for p in parameters]
    return parameter_names

parameter_names = get_parameter_names(function_signature)
parameter_names

['device', 'src', 'dst', 'scalar']

In [12]:
def get_parameter_names_with_types(function_signature):
    # split the function signature into a list of parameters
    function_signature = function_signature.replace("\n", " ")
    #function_signature = function_signature.replace("const", " ")
    #function_signature = function_signature.replace("&", " ")
    while "  " in function_signature:
        function_signature = function_signature.replace("  ", " ")
    
    parameters = function_signature[1:-1].split(',')
    return [p.strip() for p in parameters]

parameter_names = get_parameter_names_with_types(function_signature)
parameter_names

['const std::shared_ptr<::Processor> & device',
 'const Image & src',
 'const Image & dst',
 'const float & scalar']

In [13]:
def translateTypesToJava(parameter):
    parameter = parameter.replace("Image", "BufferJ")
    parameter = parameter.replace("size_t", "int")
    parameter = parameter.replace("std::shared_ptr<::Processor>", "ProcessorJ")
    return parameter
    
parameter_names_java = [translateTypesToJava(p) for p in parameter_names]
parameter_names_java

['const ProcessorJ & device',
 'const BufferJ & src',
 'const BufferJ & dst',
 'const float & scalar']

In [14]:
def generate_cpp_wrapper(function_name, tier, parameter_names):
    # generate the function name for python and c++
    py_function_name = '_' + function_name + 'Kernel_Call'
    cpp_function_name = function_name + 'Kernel_Call'
    tier_function_name = function_name[0].lower() + function_name[1:]
    # generate the function body for the wrapper
    #wrapper_function = "m.def(\"" + py_function_name + "\", &cle::" + cpp_function_name + ", \"Call " + cpp_function_name + " from C++.\",\n"
    #wrapper_function += "pybind11::arg(\""+ parameter_names[0] +"\")"
    
    parameter_names_merged = ", ".join(parameter_names)
    parameter_names_call = []
    for parameter_name in parameter_names:
        name = parameter_name.split(" ")[-1]
        if "ProcessorJ" in parameter_name:
            parameter_names_call.append(name + ".getShared()")
        elif "BufferJ" in parameter_name:
            parameter_names_call.append(name + ".get()")
        else:
            parameter_names_call.append(name)
    parameter_names_call_merged = ", ".join(parameter_names_call)
    
    cpp_wrapper = f"""
    void Tier{tier}::{tier_function_name}({parameter_names_merged})
    {{
        cle::{cpp_function_name}({parameter_names_call_merged});
    }}
    """
    return cpp_wrapper

cpp_wrapper = generate_cpp_wrapper(function_name, tier, parameter_names_java)
print(cpp_wrapper)


    void Tier1::divideImageAndScalar(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, const float & scalar)
    {
        cle::DivideImageAndScalarKernel_Call(device.getShared(), src.get(), dst.get(), scalar);
    }
    


In [15]:
def generate_cpp_signature(cpp_wrapper):
    signature = cpp_wrapper.split("\n")[1].replace("void", "static void")
    for tier in range(20, 0, -1):
        signature = signature.replace("Tier" + str(tier) + "::", "")
        
    x = 16
    return signature[:x] +  signature[x].lower() + signature[x+1:] + ";"

signature = generate_cpp_signature(cpp_wrapper)
print(signature)

    static void divideImageAndScalar(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, const float & scalar);


In [16]:
def camel_to_snake(camel_case):
    snake_case = ''
    for i, char in enumerate(camel_case):
        if i > 0 and char.isupper():
            snake_case += '_'
        snake_case += char.lower()
    return snake_case

camel_to_snake("DivideImageAndScalar")

'divide_image_and_scalar'

In [17]:
def generate_clij3_wrapper(function_name, tier, parameter_names):
    
    java_function_name = camel_to_snake(function_name)
    cpp_function_name = function_name[0].lower() + function_name[1:]
    
    stub_code = []
    parameters_signature = []
    parameters_call = []
    first_buffer = None
    return_buffer = None
    for parameter_name in parameter_names:
        name = parameter_name.split(" ")[-1]
        parameter_type = parameter_name.replace("const", " ").replace("&", " ").strip()
        while "  " in parameter_type:
            parameter_type = parameter_type.replace("  ", " ")
        parameter_type = parameter_type.split(" ")[0]

        if parameter_type == "ProcessorJ":
            pass
        elif parameter_type == "BufferJ":
            if first_buffer is None:
                first_buffer = name
            parameters_signature.append("Object " + name)
            parameters_call.append(name + "J")
            stub_code.append(f"BufferJ {name}J = push({name});")
            return_buffer = name
        else:
            parameters_signature.append(parameter_type + " " + name)
            parameters_call.append(name)
    
    # the creator_function depends on the function; e.g. a maximum_x_projection would create a 2D image from a 3D image
    # Todo: This might be removed after resolving https://github.com/clEsperanto/CLIc_prototype/issues/164
    creator_function = "create_like_if_none(" + first_buffer + "J, "
    if produces_label_image(function_name):
        creator_function = "create_labels_like_if_none(" + first_buffer + "J, "
    elif produces_binary_image(function_name):
        creator_function = "create_binary_like_if_none(" + first_buffer + "J, "
    elif produces_x_projection(function_name):
        creator_function = "create_2d_yz_like_if_none(" + first_buffer + "J, "
    elif produces_y_projection(function_name):
        creator_function = "create_2d_zx_like_if_none(" + first_buffer + "J, "
    elif produces_z_projection(function_name):
        creator_function = "create_2d_yx_like_if_none(" + first_buffer + "J, "
    
    if len(stub_code) > 1:
        stub_code[-1] = stub_code[-1].replace("push(", creator_function)
    
    parameters_call = ", ".join(parameters_call)
    parameters_signature = ", ".join(parameters_signature)
    stub_code = "\n        ".join(stub_code)
    
    clij3_wrapper = f"""
    public BufferJ {java_function_name}({parameters_signature}) {{
        {stub_code}
        Tier{tier}.{cpp_function_name}(processor, {parameters_call});
        return {return_buffer}J;
    }}
    """
    
    return clij3_wrapper

clij3_wrapper = generate_clij3_wrapper(function_name, tier, parameter_names_java)

print(clij3_wrapper)


    public BufferJ divide_image_and_scalar(Object src, Object dst, float scalar) {
        BufferJ srcJ = push(src);
        BufferJ dstJ = create_like_if_none(srcJ, dst);
        Tier1.divideImageAndScalar(processor, srcJ, dstJ, scalar);
        return dstJ;
    }
    


## Generate KERNELJ header for all tiers


In [18]:
def generate_tier_header_code(tier):

    # get current directory
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # get the relative path to directory and add 'tier' plus the number of the tier
    tier_dir = os.path.join(CLIC_DIR, 'include', 'tier' + str(tier))
    # list all files in the tier_dir
    files = glob.glob(tier_dir + '/*.hpp')
    files.sort()
    #print(files)
    
    list_of_functions = []
    
    for file in files:
        function_name = os.path.basename(file)[3:-10]
        function_signature = read_function_signature(file, function_name)
        parameter_names = get_parameter_names_with_types(function_signature)
        parameter_names_java = [translateTypesToJava(p) for p in parameter_names]
        if function_name not in GENERATOR_BLACKLIST:
            cpp_wrapper = generate_cpp_wrapper(function_name, tier, parameter_names_java)
            signature = generate_cpp_signature(cpp_wrapper)

            list_of_functions.append(signature)
    
    
    list_of_functions = "\n    ".join(list_of_functions)
    
    return f"""
class Tier{tier}
{{
public:
    {list_of_functions}
}};
    """

tier1_hpp = generate_tier_header_code(1)
print(tier1_hpp)
tier2_hpp = generate_tier_header_code(2)
print(tier2_hpp)
tier3_hpp = generate_tier_header_code(3)
print(tier3_hpp)
tier4_hpp = generate_tier_header_code(4)
print(tier4_hpp)
tier5_hpp = generate_tier_header_code(5)
print(tier5_hpp)
tier6_hpp = generate_tier_header_code(6)
print(tier6_hpp)



class Tier1
{
public:
        static void absolute(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst);
        static void addImageAndScalar(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, const float & value);
        static void addImagesWeighted(const ProcessorJ & device, const BufferJ & src1, const BufferJ & src2, const BufferJ & dst, const float & w1, const float & w2);
        static void binaryAnd(const ProcessorJ & device, const BufferJ & src1, const BufferJ & src2, const BufferJ & dst);
        static void binaryEdgeDetection(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst);
        static void binaryNot(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst);
        static void binaryOr(const ProcessorJ & device, const BufferJ & src1, const BufferJ & src2, const BufferJ & dst);
        static void binarySubtract(const ProcessorJ & device, const BufferJ & src1, const BufferJ & src2, const BufferJ & dst);

In [19]:
kernel_header = f"""
#ifndef __INCLUDE_KERNELJ_HPP
#define __INCLUDE_KERNELJ_HPP

#include "clesperantoj.hpp"

{tier1_hpp}
{tier2_hpp}
{tier3_hpp}
{tier4_hpp}
{tier5_hpp}
{tier6_hpp}

#endif // __INCLUDE_KERNELJ_HPP
    """

print(kernel_header[:400])


#ifndef __INCLUDE_KERNELJ_HPP
#define __INCLUDE_KERNELJ_HPP

#include "clesperantoj.hpp"


class Tier1
{
public:
        static void absolute(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst);
        static void addImageAndScalar(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, const float & value);
        static void addImagesWeighted(const ProcessorJ & d


In [20]:
with open(KERNELJ_HEADER_FILE, 'w') as f:
    f.write(kernel_header)

## Write KernelJ CPP

In [21]:
def generate_tier_cpp_code(tier):

    # get current directory
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # get the relative path to directory and add 'tier' plus the number of the tier
    tier_dir = os.path.join(CLIC_DIR, 'include', 'tier' + str(tier))
    # list all files in the tier_dir
    files = glob.glob(tier_dir + '/*.hpp')
    files.sort()
    #print(files)
    
    list_of_functions = []
    
    for file in files:
        function_name = os.path.basename(file)[3:-10]
        function_signature = read_function_signature(file, function_name)
        parameter_names = get_parameter_names_with_types(function_signature)
        parameter_names_java = [translateTypesToJava(p) for p in parameter_names]
        if function_name not in GENERATOR_BLACKLIST:
            cpp_wrapper = generate_cpp_wrapper(function_name, tier, parameter_names_java)

            list_of_functions.append(cpp_wrapper)
    
    
    list_of_functions = "\n    ".join(list_of_functions)
    
    return list_of_functions

tier1_cpp = generate_tier_cpp_code(1)
print(tier1_cpp[:300])
tier2_cpp = generate_tier_cpp_code(2)
print(tier2_cpp[:300])

tier3_cpp = generate_tier_cpp_code(3)
tier4_cpp = generate_tier_cpp_code(4)
tier5_cpp = generate_tier_cpp_code(5)
tier6_cpp = generate_tier_cpp_code(6)


    void Tier1::absolute(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst)
    {
        cle::AbsoluteKernel_Call(device.getShared(), src.get(), dst.get());
    }
    
    
    void Tier1::addImageAndScalar(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, cons

    void Tier2::dilateLabels(const ProcessorJ & device, const BufferJ & src, const BufferJ & dst, const float & radius)
    {
        cle::DilateLabelsKernel_Call(device.getShared(), src.get(), dst.get(), radius);
    }
    
    
    void Tier2::extendLabelingViaVoronoi(const ProcessorJ & device, c


In [22]:
kernel_cpp_code = f"""
#include "kernelj.hpp"

#include "cleKernelList.hpp"

{tier1_cpp}
{tier2_cpp}
{tier3_cpp}
{tier4_cpp}
{tier5_cpp}
{tier6_cpp}
"""

with open(KERNELJ_SOURCE_FILE, 'w') as f:
    f.write(kernel_cpp_code)


## Write CLIJ3 gateway

In [23]:
def generate_clij3_gateway_code(tier):

    # get current directory
    current_dir = os.path.dirname(os.path.abspath(__file__))
    # get the relative path to directory and add 'tier' plus the number of the tier
    tier_dir = os.path.join(CLIC_DIR, 'include', 'tier' + str(tier))
    # list all files in the tier_dir
    files = glob.glob(tier_dir + '/*.hpp')
    files.sort()
    #print(files)
    
    list_of_functions = []
    
    for file in files:
        function_name = os.path.basename(file)[3:-10]
        function_signature = read_function_signature(file, function_name)
        parameter_names = get_parameter_names_with_types(function_signature)
        parameter_names_java = [translateTypesToJava(p) for p in parameter_names]
        if function_name not in GENERATOR_BLACKLIST:
            #print(function_name)
            java_wrapper = generate_clij3_wrapper(function_name, tier, parameter_names_java)

            list_of_functions.append(java_wrapper)
    
    
    list_of_functions = "\n    ".join(list_of_functions)
    
    return list_of_functions

tier1_java = generate_clij3_gateway_code(1)
print(tier1_java[:200])

tier2_java = generate_clij3_gateway_code(2)
print(tier2_java[:200])

tier3_java = generate_clij3_gateway_code(3)
tier4_java = generate_clij3_gateway_code(4)
tier5_java = generate_clij3_gateway_code(5)
tier6_java = generate_clij3_gateway_code(6)



    public BufferJ absolute(Object src, Object dst) {
        BufferJ srcJ = push(src);
        BufferJ dstJ = create_like_if_none(srcJ, dst);
        Tier1.absolute(processor, srcJ, dstJ);
        r

    public BufferJ dilate_labels(Object src, Object dst, float radius) {
        BufferJ srcJ = push(src);
        BufferJ dstJ = create_labels_like_if_none(srcJ, dst);
        Tier2.dilateLabels(pro


In [24]:
with open(CLIJ3_GATEWAY_SOURCE_FILE, 'r') as file:
    # Read the entire contents of the file
    clij3_gateway_code = file.read()

In [25]:
separator = "// GENERATOR //"
temp = clij3_gateway_code.split(separator)
temp[1] = tier1_java + "\n\n" + tier2_java + "\n\n" + tier3_java + "\n\n" + tier4_java + "\n\n" + tier5_java + "\n\n" + tier6_java

clij3_gateway_code = separator.join(temp)

with open(CLIJ3_GATEWAY_SOURCE_FILE, 'w') as f:
    f.write(clij3_gateway_code)
